# Wellahh Food Recommendation System

Reference: T. N. Trang Tran, M. Atas, A. Felfernig, and M. Stettinger, “An overview of recommender systems in the healthy food domain,” J. Intell. Inf. Syst., vol. 50, no. 3, pp. 501–526, Jun. 2018, doi: 10.1007/s10844-017-0469-0.

- download dulu datasetnya bang: https://drive.google.com/uc?export=download&id=1zPr5XKMSAeaeY6IKQWnABYmwzlYUBNhU

In [128]:
import urllib
import pandas as pd
import pandas as pd
import numpy as np

from sklearn import preprocessing

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import re
import string
from pathlib import Path
from nltk.stem.snowball import SnowballStemmer


# my_file_2 = Path("./recipes.csv")
# if my_file_2.is_file() == False:
#     url = "https://drive.google.com/uc?export=download&id=1zPr5XKMSAeaeY6IKQWnABYmwzlYUBNhU"
#     path = "recipes.csv"
#     urllib.request.urlretrieve(url, path)
path2 = "./recipes.csv"



df2 = pd.read_csv(path2)


# Data Prep

In [129]:
df2.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil..."


In [130]:
df2 = df2.drop_duplicates(subset=['Name'], keep="last")


In [131]:
df2= df2.reset_index(drop=True)

In [132]:
df2 = df2.drop([idx for idx in range(10000, len(df2))])

In [133]:
df2= df2.reset_index(drop=True)

In [134]:
# df2.to_csv('recipes_sedikit.csv')


In [135]:
for idx,row in df2.iterrows():
     df2.iloc[idx:idx+1, 13:14] = row["RecipeIngredientParts"] .replace('c(', '').replace(')', '').replace('"', '').replace(',', '')


In [136]:
df2["RecipeIngredientParts"]

0       blueberries granulated sugar vanilla yogurt le...
1       extra firm tofu eggplant zucchini mushrooms so...
2       graham cracker crumbs sugar butter sugar corns...
3       chicken butter flour milk celery button mushro...
4       sugar margarine egg flour salt buttermilk grah...
                              ...                        
9995    margarine chocolate chips mini marshmallows co...
9996       heavy cream unsalted butter white sugar cognac
9997                                      white chocolate
9998    shrimp plain yogurt fresh lemon juice cayenne ...
9999    napa cabbage olive oil onion soy sauce fresh l...
Name: RecipeIngredientParts, Length: 10000, dtype: object

# Content Based Filtering
- merekomendasikan makanan yang relevan dengan beberapa makanan favorit pengguna,
- relevansi berdasarkan ingredient makanan, nama makanan, dan category makanan


In [137]:

def create_features(x):
    return x['RecipeIngredientParts'] + " " + x['RecipeCategory'] + " " + x["Name"]
    

In [138]:
features = ['RecipeIngredientParts', 'RecipeCategory']


In [139]:
df2['soup'] = df2.apply(create_features, axis=1)

In [140]:
count = CountVectorizer()
count_matrix = count.fit_transform(df2['soup'])

In [141]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [142]:
df2 = df2.reset_index(drop=True)


In [143]:
def name_tf_idf(x):
    return  x['Name'] 
    

In [144]:
df2['soup_name'] = df2.apply(name_tf_idf, axis=1)

In [145]:
df2.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,soup,soup_name
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan...",blueberries granulated sugar vanilla yogurt le...,Low-Fat Berry Blue Frozen Dessert
1,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc...",extra firm tofu eggplant zucchini mushrooms so...,Carina's Tofu-Vegetable Kebabs
2,43,Best Blackbottom Pie,34879,Barefoot Beachcomber,PT2H,PT20M,PT2H20M,1999-08-21T10:35:00Z,Make and share this Best Blackbottom Pie recip...,character(0),...,267.6,58.0,1.8,42.5,7.0,8.0,1 9-inch pie,"c(""Graham Cracker Crust: In small bowl, combin...",graham cracker crumbs sugar butter sugar corns...,Best Blackbottom Pie
3,44,Warm Chicken A La King,1596,Joan Edington,PT3M,PT35M,PT38M,1999-09-17T04:47:00Z,I copied this one out of a friend's book so ma...,"""https://img.sndimg.com/food/image/upload/w_55...",...,557.2,29.1,3.1,5.0,45.3,2.0,NaN,"c(""Melt 1 1/2 ozs butter, add the flour and co...",chicken butter flour milk celery button mushro...,Warm Chicken A La King
4,45,Buttermilk Pie With Gingersnap Crumb Crust,1580,tristitia,PT50M,PT30M,PT1H20M,1999-08-06T00:40:00Z,Make and share this Buttermilk Pie With Ginger...,"""https://img.sndimg.com/food/image/upload/w_55...",...,281.8,37.5,0.5,24.7,4.2,8.0,NaN,"c(""Preheat oven to 350°F."", ""Make pie crust, u...",sugar margarine egg flour salt buttermilk grah...,Buttermilk Pie With Gingersnap Crumb Crust


In [146]:
count_matrix_name = count.fit_transform(df2['soup_name'])

In [147]:
indices = pd.Series(df2.index, index=df2['Name'])

In [148]:
def show_best_results(df, scores_array, top_n=1):
  sorted_indices = scores_array.argsort()[::-1]
  for position, idx in enumerate(sorted_indices[:top_n]):
    row = df.iloc[idx]
    food_name = row["Name"]
    score = scores_array[idx]
    return food_name

def query_food_name(query):
    query_vectorized = count.transform([query])
    scores = query_vectorized.dot(count_matrix_name.transpose())
    scores_array = scores.toarray()[0]
    return show_best_results(df2, scores_array)

In [149]:
query = "fried chicken"
query_vectorized = count.transform([query])
scores = query_vectorized.dot(count_matrix_name.transpose())
scores_array = scores.toarray()[0]
show_best_results(df2, scores_array)

'Japanese Chicken Karaage (Fried Chicken)'

In [150]:
def get_recommendations(food_name, cosine_sim=cosine_sim):
    
    matched_food_name = query_food_name(food_name)
    idx = indices[matched_food_name]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    if len(sim_scores) > 49:
        sim_scores = sim_scores[1:51]
    else:
        sim_scores =  sim_scores[1:len(sim_scores)]
        
    food_indices = [i[0] for i in sim_scores]
    # return df2['Name'].iloc[food_indices]
    return df2.iloc[food_indices]

In [151]:
get_recommendations("fried chicken")

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,soup,soup_name
1614,3929,Oven Fried Chicken II,1886,HELEN PEAGRAM,PT40M,PT10M,PT50M,1999-11-03T07:10:00Z,Make and share this Oven Fried Chicken II reci...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,623.0,12.3,0.6,0.1,15.3,4.0,NaN,"c(""Preheat oven to 450"", ""Combine all dry ingr...",chicken parts flour salt fresh pepper paprika ...,Oven Fried Chicken II
2206,4941,Katherine's Fried Chicken,1534,Tonkcats,PT1H30M,PT0S,PT1H30M,1999-12-01T20:03:00Z,Make and share this Katherine's Fried Chicken ...,character(0),...,84.0,35.2,6.1,0.7,28.1,8.0,NaN,"c(""In a large bowl add flour, pepper, seasoned...",whole wheat flour black pepper seasoning salt ...,Katherine's Fried Chicken
114,236,Chicken Curry II,1597,Kathy Bloor,NaN,PT1H,PT1H,1999-08-12T06:14:00Z,"This is a very mild chicken curry, very suitab...",character(0),...,544.8,17.3,1.0,3.5,26.1,8.0,NaN,"c(""The day before, place whole chicken in a la...",chicken unsalted butter onions salt flour curr...,Chicken Curry II
8671,15740,Pecan-Crusted Chicken Drumsticks,20754,RecipeNut,PT45M,PT15M,PT1H,2001-12-13T09:25:00Z,Make and share this Pecan-Crusted Chicken Drum...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,947.8,7.3,0.8,2.9,60.5,6.0,NaN,"c(""PREHEAT oven to 400°E Toss chicken legs wit...",chicken drumsticks chicken legs egg pecans sug...,Pecan-Crusted Chicken Drumsticks
8554,15583,Crispy Coated Garlic Chicken,21752,Mirj2338,PT1H30M,PT8H,PT9H30M,2001-12-11T11:22:00Z,Very delicious and garlicky! Share this with s...,character(0),...,370.6,8.8,0.5,0.8,85.4,8.0,NaN,"c(""Place chicken in a large, shallow container...",chicken olive oil garlic garlic powder salt pe...,Crispy Coated Garlic Chicken
1958,4510,Fried Chicken Livers II,1639,elt8588,NaN,PT0S,PT0S,1999-11-15T04:26:00Z,Make and share this Fried Chicken Livers II re...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,3203.7,108.2,3.4,0.7,104.2,NaN,1 batch,"c(""Mix milk and egg; set aside. Mix dry ingre...",chicken liver milk egg baking powder salt flou...,Fried Chicken Livers II
3754,8350,Crispy Chicken Drumsticks,82616,Shirl J 831,PT1H,PT10M,PT1H10M,2000-03-16T20:09:00Z,Make and share this Crispy Chicken Drumsticks ...,character(0),...,1020.6,32.5,2.2,2.7,37.8,4.0,NaN,"c(""Rinse chicken with cold water, and pat dry....",chicken drumsticks parmesan cheese garlic powd...,Crispy Chicken Drumsticks
4044,8782,"Roast ""Sticky"" Chicken",6411,C Chatwin,PT5H10M,PT15M,PT5H25M,2001-04-20T18:06:00Z,"Beautiful and delicious, this incredibly moist...","c(""https://img.sndimg.com/food/image/upload/w_...",...,1619.9,6.2,1.4,2.3,17.6,6.0,2 Servings per Lb,"c(""(Day 1) Blend all spices together and set a...",salt paprika cayenne pepper onion powder thyme...,"Roast ""Sticky"" Chicken"
1554,3842,Real Fried Chicken,1859,Robb114,PT2H,PT20M,PT2H20M,1999-11-01T07:29:00Z,There's only one right way to fry chicken and ...,character(0),...,4811.0,47.9,1.8,0.2,48.7,NaN,NaN,"c(""In a shallow bowl, combine flour, salt and ...",chicken flour salt pepper butter shortening Wh...,Real Fried Chicken
7053,13440,Easy Chicken Crescents,20893,Brenda,PT15M,PT15M,PT30M,2001-10-29T16:52:00Z,Make and share this Easy Chicken Crescents rec...,character(0),...,1649.1,65.7,4.6,5.9,29.2,6.0,12 crescents (approx),"c(""Preheat oven to 350°F."", ""Combine chicken, ...",cream cheese onion powder garlic powder season...,Easy Chicken Crescents


In [118]:
get_recommendations('burger')

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,soup,soup_name
3700,8262,Lemon Chicken II,123897,Vnut-Beyond Redempt,PT40M,PT6H,PT6H40M,2000-03-13T21:51:00Z,Make and share this Lemon Chicken II recipe fr...,character(0),...,293.4,9.6,0.1,7.4,25.8,6.0,NaN,"c(""Combine chicken and lemon juice."", ""Marinat...",chicken pieces lemon juice lemon juice salt br...,Lemon Chicken II
9526,16940,Zurichgeschnetzeltes-or-diced chicken,10404,Diana Adcock,PT6M,PT5M,PT11M,2002-01-08T09:43:00Z,Make and share this Zurichgeschnetzeltes-or-di...,character(0),...,174.4,4.3,0.7,1.5,55.1,2.0,NaN,"c(""Heat oil in a medium fry pan over medium hi...",chicken breasts mushrooms lemon juice white wi...,Zurichgeschnetzeltes-or-diced chicken
1903,4416,Chicken Tart,1543,Doreen Randal,PT20M,PT15M,PT35M,1999-11-12T09:54:00Z,Make and share this Chicken Tart recipe from F...,character(0),...,70.7,22.2,0.8,0.1,9.8,4.0,NaN,"c(""Sift the flour and a pinch of salt into a b...",flour butter chicken lemon juice dried tarrago...,Chicken Tart
4258,9106,Curried Chicken and Peach Salad,2312,Gay Gilmore,PT5M,PT15M,PT20M,2001-05-25T14:27:00Z,Make and share this Curried Chicken and Peach ...,character(0),...,204.5,23.6,4.2,16.3,4.8,4.0,NaN,"c(""Combine the first 8 ingredients."", ""Add the...",fat-free mayonnaise fat free sour cream sugar ...,Curried Chicken and Peach Salad
6197,12197,Souper Chicken,13483,Tebo3759,PT2H30M,PT5M,PT2H35M,2001-09-28T11:54:00Z,Got a crowd coming over? This can serve a lot ...,"""https://img.sndimg.com/food/image/upload/w_55...",...,1023.2,9.1,0.7,2.5,28.6,NaN,NaN,"c(""Preheat oven to 300°F."", ""Mix soups and win...",dry white wine chicken pieces Chicken Souper C...,Souper Chicken
3012,7208,Old Fashioned Chicken Salad,34879,Barefoot Beachcomber,PT5M,PT30M,PT35M,2000-03-06T16:24:00Z,Make and share this Old Fashioned Chicken Sal...,character(0),...,149.0,31.3,2.9,24.9,5.0,10.0,NaN,"c(""Oil a 10\"" X 13\"" pan."", ""Soften the gelati...",unflavored gelatin chicken broth lemon juice t...,Old Fashioned Chicken Salad
9436,16814,Tandoori Chicken Breasts,21399,Karen From Colorado,PT12M,PT1H30M,PT1H42M,2002-01-06T07:56:00Z,Make and share this Tandoori Chicken Breasts r...,character(0),...,102.6,2.3,0.2,1.2,31.2,4.0,NaN,"c(""Lightly score chicken breasts 3 or 4 times ...",chicken breasts lemon juice plain low-fat yogu...,Tandoori Chicken Breasts
1095,3106,Chicken With Honey Lemon,123897,Vnut-Beyond Redempt,PT50M,PT1H,PT1H50M,1999-10-01T22:50:00Z,Make and share this Chicken With Honey Lemon r...,character(0),...,2333.1,70.5,1.4,65.9,0.8,NaN,1 spring chicken,"c(""Mix oil, lemon juice, pepper, honey, salt a...",lemon juice pepper honey salt brown sugar spri...,Chicken With Honey Lemon
7918,14670,Rice Crispy Chicken,11295,ThatJodiGirl,PT1H,PT15M,PT1H15M,2001-11-25T13:34:00Z,"This is my moms recipe, it is simple and soooo...","""https://img.sndimg.com/food/image/upload/w_55...",...,720.2,35.8,0.3,4.0,24.4,NaN,9 pieces,"c(""Place rice crispies in large ziploc bag, cr...",butter chicken Chicken Rice Crispy Chicken,Rice Crispy Chicken
2170,4882,Hungarian Chicken Paprika,1957,Sam29839,NaN,PT0S,PT0S,1999-11-24T06:34:00Z,"This is a recipe for Chicken Paprika, a recip...","""https://img.sndimg.com/food/image/upload/w_55...",...,1067.1,24.3,1.4,4.8,54.4,6.0,NaN,"c(""Sauce procedure: Cut whole chicken into it'...",chicken sour cream chicken thighs paprika salt...,Hungarian Chicken Paprika


In [152]:
get_recommendations('steak')

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,soup,soup_name
8091,14912,5 Star Gourmet Sauce for Steak,10649,Ben Ross,PT10M,PT5M,PT15M,2001-11-27T09:20:00Z,This sauce was like heaven had come to my lips...,character(0),...,435.5,9.2,3.0,6.0,2.2,2.0,NaN,"c(""Place the oil in the bottom of a saucepan.""...",diced tomato salt pepper dry sherry tomato sau...,5 Star Gourmet Sauce for Steak
4464,9430,Simmered Round Steak,11044,jean1490,PT2H,PT20M,PT2H20M,2001-06-15T09:20:00Z,Make and share this Simmered Round Steak recip...,"""https://img.sndimg.com/food/image/upload/w_55...",...,792.7,9.9,1.1,7.5,24.4,NaN,NaN,"c(""Trim fat from steak and cut meat into servi...",onion sugar salt Worcestershire sauce prepared...,Simmered Round Steak
2930,7057,Ranch Round Steak,87236,truebrit,PT1H30M,PT10M,PT1H40M,2000-03-06T16:21:00Z,Make and share this Ranch Round Steak recipe f...,character(0),...,457.4,3.6,0.2,0.3,0.5,8.0,NaN,"c(""Trim excess fat from meat; slash edges to p...",beef round steak flour dry mustard salt pepper...,Ranch Round Steak
4594,9742,Steak with Caramelized Onions,10420,mielhollinger,PT20M,PT5M,PT25M,2001-06-29T10:52:00Z,Caramelized onions splashed with balsamic vine...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,174.5,9.8,1.0,6.3,25.9,4.0,NaN,"c(""In a medium skillet melt butter over medium...",butter onions garlic brown sugar salt boneless...,Steak with Caramelized Onions
9403,16766,Japanese Steak With Sunomono,21399,Karen From Colorado,PT15M,PT45M,PT1H,2002-01-05T14:26:00Z,Sunomono (pronounced soo-noh-moh-noh) is Japan...,character(0),...,1424.2,9.9,1.5,6.0,43.9,4.0,NaN,"c(""To prepare sunomono----------------."", ""Pla...",cucumber salt sugar radish carrot soy sauce dr...,Japanese Steak With Sunomono
1488,3742,Pacific Rim Glazed Flank Steak,1533,Dancer,NaN,PT0S,PT0S,1999-10-28T19:17:00Z,Make and share this Pacific Rim Glazed Flank S...,character(0),...,2852.3,38.8,0.6,35.9,51.6,NaN,NaN,"c(""In medium shallow dish, combine marinade in...",beef flank steak onion honey fresh rosemary ga...,Pacific Rim Glazed Flank Steak
9036,16273,"Karen's Swiss Steak (Stove Top, Crock Pot or O...",21399,Karen From Colorado,PT1H15M,PT30M,PT1H45M,2001-12-26T12:41:00Z,"This recipe can be prepared on the stove top, ...","c(""https://img.sndimg.com/food/image/upload/w_...",...,440.8,11.3,2.1,4.0,1.8,6.0,NaN,"c(""Cut meat into 6 serving pieces."", ""Combine ...",beef round steak flour salt dry mustard pepper...,"Karen's Swiss Steak (Stove Top, Crock Pot or O..."
6960,13286,Carpetbagger Steak,10113,southern chef in lo,PT20M,PT15M,PT35M,2001-10-26T15:40:00Z,Make and share this Carpetbagger Steak recipe ...,character(0),...,303.4,3.7,0.1,0.4,8.2,4.0,NaN,"c(""Sauté oysters, mushrooms, and parsley in th...",fresh mushrooms fresh parsley butter bacon blu...,Carpetbagger Steak
4733,9965,Salisbury Steak with Egg Noodles,10426,Laura Bruce,PT17M,PT0S,PT17M,2001-07-04T11:19:00Z,Make and share this Salisbury Steak with Egg N...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,788.4,41.6,2.0,6.7,30.0,6.0,NaN,"c(""In a large bowl, beat egg."", ""Stir in 1/3 c...",egg salt pepper ground beef all-purpose flour ...,Salisbury Steak with Egg Noodles
3744,8334,Mushroom Stuffed Flank Steak,39547,Julesong,NaN,PT0S,PT0S,2000-03-16T20:09:00Z,Make and share this Mushroom Stuffed Flank Ste...,character(0),...,721.3,12.4,1.6,3.7,35.6,NaN,NaN,"c(""Lightly salt and pepper both sides of the f...",salt white pepper Dijon mustard onion mushroom...,Mushroom Stuffed Flank Steak


In [153]:
get_recommendations('Ramen')

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,soup,soup_name
2957,7099,Steak-Vegetable Pockets,33443,Nat Da Brat,NaN,PT0S,PT12M,2000-03-06T16:22:00Z,Make and share this Steak-Vegetable Pockets re...,character(0),...,1152.4,45.1,4.5,5.6,29.5,4.0,NaN,"c(""Thinly slice beef into bite-size strips."", ...",beef top round steak broccoli carrot onion gre...,Steak-Vegetable Pockets
771,2632,Stir-Fry Beef & Spinach with Noodles,1533,Dancer,PT30M,PT0S,PT30M,1999-08-13T13:50:00Z,Make and share this Stir-Fry Beef & Spinach wi...,character(0),...,599.9,54.4,4.2,8.0,12.8,4.0,NaN,"c(""Stack beef steaks; cut lengthwise in half a...",beef round tip steak fresh spinach water chest...,Stir-Fry Beef & Spinach with Noodles
9812,17354,Beef Bundle Kabobs,21399,Karen From Colorado,PT10M,PT3H,PT3H10M,2002-01-17T17:14:00Z,Wonderful easy kabobs done on the grill. You c...,character(0),...,1471.7,14.4,4.1,6.6,26.3,NaN,NaN,"c(""Combine soy sauce, sugar and ginger in a me...",soy sauce sugar ground ginger beef round tip s...,Beef Bundle Kabobs
2930,7057,Ranch Round Steak,87236,truebrit,PT1H30M,PT10M,PT1H40M,2000-03-06T16:21:00Z,Make and share this Ranch Round Steak recipe f...,character(0),...,457.4,3.6,0.2,0.3,0.5,8.0,NaN,"c(""Trim excess fat from meat; slash edges to p...",beef round steak flour dry mustard salt pepper...,Ranch Round Steak
2945,7081,Stir-Fried Beef Salad,86072,Doglover61aka Earnh,NaN,PT0S,PT6M,2000-03-06T16:22:00Z,Make and share this Stir-Fried Beef Salad reci...,character(0),...,634.9,12.7,3.7,5.7,4.9,4.0,NaN,"c(""Partially freeze beef; slice thinly across ...",beef round steak garlic mushrooms cucumber gre...,Stir-Fried Beef Salad
7059,13448,Steak and Vegetable Stir-Fry,10113,southern chef in lo,PT13M,PT20M,PT33M,2001-10-29T16:53:00Z,Make and share this Steak and Vegetable Stir-F...,character(0),...,1209.8,69.8,3.9,5.4,23.8,4.0,NaN,"c(""In a bowl, combine the cornstarch, bouillon...",cornstarch beef bouillon granules water soy sa...,Steak and Vegetable Stir-Fry
2997,7183,Beef Stroganoff III,87236,truebrit,NaN,PT0S,PT0S,2000-03-06T16:24:00Z,Make and share this Beef Stroganoff III recipe...,"""https://img.sndimg.com/food/image/upload/w_55...",...,649.9,9.8,0.9,6.0,3.9,6.0,NaN,"c(""Cook onion in butter until transparent."", ""...",beef round steak soy sauce ketchup onion flour...,Beef Stroganoff III
513,1267,Chinese Beef and Broccoli,1762,lkadlec,PT10M,PT35M,PT45M,1999-09-26T20:31:00Z,Make and share this Chinese Beef and Broccoli ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,592.4,7.1,3.5,1.8,28.3,4.0,NaN,"c(""Slice the steak against the grain into very...",soy sauce dry sherry cornstarch frozen broccol...,Chinese Beef and Broccoli
1737,4137,Beef Jerky II,1903,tcantwell,NaN,PT0S,PT0S,1999-11-07T13:36:00Z,Make and share this Beef Jerky II recipe from ...,character(0),...,6019.2,22.4,0.9,11.5,8.1,NaN,1 batch,"c(""Remove fat from meat and place [meat] in fr...",boneless round steak soy sauce Worcestershire ...,Beef Jerky II
9695,17187,Beef Round Steak Roll Up,3288,TishT,PT6H,PT20M,PT6H20M,2002-01-14T09:21:00Z,You can make the roll ups the night before and...,character(0),...,127.2,2.5,0.7,1.2,35.7,6.0,NaN,"c(""Cut the meat into 6 pieces and pound each p...",boneless beef round steak carrot zucchini gree...,Beef Round Steak Roll Up


In [154]:
get_recommendations('pork')

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,soup,soup_name
9663,17140,Grilled Pork Loin With Bacon,25455,SharleneW,PT1H,PT10M,PT1H10M,2002-01-12T17:39:00Z,Make and share this Grilled Pork Loin With Bac...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,239.5,0.6,0.1,0.0,46.6,6.0,NaN,"c(""Chop bacon. Mince garlic."", ""Crush rosemary...",bacon garlic cloves dried rosemary boneless po...,Grilled Pork Loin With Bacon
8916,16098,"Pork Scalloppini With Green Olives, Tomato And...",13063,skat5762,PT15M,PT5M,PT20M,2001-12-21T09:34:00Z,Make and share this Pork Scalloppini With Gree...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,343.7,5.4,0.7,1.8,17.0,4.0,NaN,"c(""Lay the pork in a single layer on a plate a...",boneless pork flour olive oil garlic shallot r...,"Pork Scalloppini With Green Olives, Tomato And..."
1018,2987,Spicy Pork and Black Bean Chili,1544,tranch,NaN,PT0S,PT0S,1999-08-19T05:54:00Z,Make and share this Spicy Pork and Black Bean ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,163.6,24.9,7.8,4.8,24.4,8.0,NaN,"c(""In a large pot, cover beans with water and ...",black beans boneless lean pork garlic paprika ...,Spicy Pork and Black Bean Chili
8594,15635,"Oven-Baked Pork Loin With Olive Oil, Garlic, T...",10420,mielhollinger,PT3H,PT24H,PT27H,2001-12-11T13:00:00Z,Make and share this Oven-Baked Pork Loin With ...,character(0),...,433.2,1.1,0.4,0.0,56.1,8.0,NaN,"c(""In a small bowl, combine the oil, garlic, t...",olive oil garlic cloves dried thyme dried oreg...,"Oven-Baked Pork Loin With Olive Oil, Garlic, T..."
4451,9409,Easy Braised Pork Chops,8119,Karen Parnell,PT47M,PT10M,PT57M,2001-06-11T13:33:00Z,This recipe creates very juicy tender pork cho...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,111.8,5.4,0.9,2.3,41.9,4.0,NaN,"c(""Season pork chops with salt and pepper."", ""...",olive oil onions garlic salt pepper Pork Easy ...,Easy Braised Pork Chops
3049,7271,Roast Pork Stuffed With Rice and Vegetables,34879,Barefoot Beachcomber,PT3H,PT30M,PT3H30M,2000-03-06T16:26:00Z,Make and share this Roast Pork Stuffed With ...,character(0),...,94.7,3.5,0.7,2.6,33.0,15.0,NaN,"c(""Cook rice according to package directions. ...",water green onions mushroom green pepper butte...,Roast Pork Stuffed With Rice and Vegetables
6141,12116,Onion Cups,14175,Michelle Bakel,PT30M,PT20M,PT50M,2001-09-26T10:28:00Z,This is a wonderfully light side dish FULL of ...,character(0),...,182.2,15.6,2.7,8.8,3.0,8.0,NaN,"c(""Combine sour cream and horseradish; chill u...",sour cream horseradish red onions olive oil sa...,Onion Cups
97,199,Citrus Glazed Barbecued Pork Loin,39547,Julesong,PT35M,PT15M,PT50M,1999-08-07T16:33:00Z,Make and share this Citrus Glazed Barbecued Po...,"""https://img.sndimg.com/food/image/upload/w_55...",...,3187.2,47.0,2.6,30.9,46.7,6.0,NaN,"c(""Preheat oven to 350 degrees."", ""In a small ...",canola oil onion garlic chipotle pepper arbol ...,Citrus Glazed Barbecued Pork Loin
3690,8251,Irish Loin of Pork with Lemon and Herbs,35635,_Pixie_,NaN,PT0S,PT3H32M,2000-03-13T21:51:00Z,Make and share this Irish Loin of Pork with Le...,character(0),...,178.7,4.6,0.6,1.3,67.5,8.0,NaN,"c(""Pat pork dry."", ""Score well with sharp knif...",boneless pork loin parsley onion lemon rind of...,Irish Loin of Pork with Lemon and Herbs
5695,11441,"Orecchiette With Spinach, Garlic and Bacon",16459,Elizabeth Fullerton,PT25M,PT20M,PT45M,2001-09-05T16:44:00Z,"Make and share this Orecchiette With Spinach,...",character(0),...,173.9,45.6,3.2,1.7,11.1,8.0,NaN,"c(""Cook pasta according to directions on the b...",frozen spinach bacon onion garlic cloves olive...,"Orecchiette With Spinach, Garlic and Bacon"


In [155]:
df2.RecipeCategory.unique()


array(['Frozen Desserts', 'Soy/Tofu', 'Pie', 'Chicken', 'Vegetable',
       'Chicken Breast', 'Dessert', 'Beverages', 'Southwestern U.S.',
       'Stew', 'Lactose Free', 'Weeknight', 'Yeast Breads', 'Cheesecake',
       'Sauces', 'High In...', 'Brazilian', 'Brown Rice', 'Oranges',
       'Free Of...', 'Low Protein', 'Potato', 'Halibut', 'Lamb/Sheep',
       'Breads', 'Spaghetti', 'Lunch/Snacks', 'Beans', 'Very Low Carbs',
       'Pineapple', 'Whole Chicken', 'Low Cholesterol', '< 30 Mins',
       'Chicken Livers', 'Coconut', '< 60 Mins', 'Poultry',
       'Quick Breads', 'Steak', 'Healthy', 'Pork', 'Scones', 'Lobster',
       'Rice', 'Punch Beverage', 'Drop Cookies', 'Spreads', 'Bar Cookie',
       'Crab', 'Pears', 'Cheese', 'Savory Pies', 'Breakfast', 'Chowders',
       'Candy', 'Chutneys', 'White Rice', 'Tex Mex', 'German', 'Meat',
       'Fruit', 'European', 'Smoothies', 'Hungarian', 'Onions',
       'New Zealand', 'Indonesian', 'Lentil', 'Summer', 'Long Grain Rice',
       'Southwe

# Filter Recommendation Results With Constraint Satisfaction Modelling
- memfilter rekomendasi makanan dari content-based filtering degnan csp modelling.
- difilter berdasarkan kandungan nutrisi makanan disesuakian dengan penyakit pengguna,umur,weight, height, dan category makanan
- solve csp model nya pakai optimized backtracking (https://web.stanford.edu/class/cs227/Lectures/lec14.pdf)
- hasilnya berupa meal plan breakfast, lunch, dan dinner

In [156]:
from constraint import *
problem = Problem()

def calorie_intake(w, h, a):
    cal =  10*w + 6.25*h - 5*a +5
    return cal

my_favorite_food = ["salad", "fried chicken", "coconut", "pizza"]
def get_all_food_recommendations(my_favorite_food):
    foods_recommendation = []
    for myfood in my_favorite_food:
        food_recommendations_from_cbf = get_recommendations("pork")
        food_recommendations_from_cbf = food_recommendations_from_cbf.to_dict('records')
        for new_food in food_recommendations_from_cbf:
            foods_recommendation.append(new_food)
    return foods_recommendation


foods_recommendation = get_all_food_recommendations(my_favorite_food)

# for i in range(0, 5):
#     print(foods_recommendation[i])
#     print()


#https://www.calculator.net/bmr-calculator.html
#BMR = 10W + 6.25H - 5A + 5

problem.addVariable("breakfast", foods_recommendation
       
                )

problem.addVariable( "lunch",  foods_recommendation
                )

problem.addVariable( "dinner",  foods_recommendation
                )


def filter_based_on_disease(breakfast, lunch, dinner, calorie_intake, disease, category):
    """
    https://journal.ipb.ac.id/index.php/jtep/article/view/52452/27802
    """
    
    saturated_fat =   breakfast["SaturatedFatContent"] + lunch["SaturatedFatContent"] + dinner["SaturatedFatContent"]
    fat =   breakfast["FatContent"] + lunch["FatContent"] + dinner["FatContent"]
    mufa_pufa = fat - saturated_fat
    
    makanan_bervariasi = breakfast != lunch and lunch != dinner and breakfast  != dinner
    match_category = breakfast["RecipeCategory"] == category and lunch["RecipeCategory"] == category and dinner["RecipeCategory"] == category
    
    if disease == "diabetes":
        max_fat = breakfast["FatContent"] + lunch["FatContent"] + dinner["FatContent"] < 0.25 * calorie_intake
        max_protein = breakfast["ProteinContent"] + lunch["ProteinContent"] + dinner["ProteinContent"] < 0.15 * calorie_intake
        max_carbo = breakfast["CarbohydrateContent"] + lunch["CarbohydrateContent"] + dinner["CarbohydrateContent"] < 0.60 * calorie_intake
        max_sodium = breakfast["SodiumContent"] + lunch["SodiumContent"] + dinner["SodiumContent"] < 2000
        max_cholesterol = breakfast["CholesterolContent"] + lunch["CholesterolContent"] + dinner["CholesterolContent"] < 200
        max_saturated_fat = saturated_fat < 0.07 * calorie_intake
        return max_fat and max_protein and max_carbo and max_sodium and max_cholesterol and max_saturated_fat  and makanan_bervariasi
        
    elif disease == "cardiovascular":
        max_fat = breakfast["FatContent"] + lunch["FatContent"] + dinner["FatContent"] < 0.25 * calorie_intake
        max_protein =  breakfast["ProteinContent"] + lunch["ProteinContent"] + dinner["ProteinContent"] < 0.15 * calorie_intake
        max_carbo = breakfast["CarbohydrateContent"] + lunch["CarbohydrateContent"] + dinner["CarbohydrateContent"] < 0.60 * calorie_intake
        max_sodium = breakfast["SodiumContent"] + lunch["SodiumContent"] + dinner["SodiumContent"] < 2000
        max_cholesterol = breakfast["CholesterolContent"] + lunch["CholesterolContent"] + dinner["CholesterolContent"] < 200
        max_saturated_fat = saturated_fat < 0.07 * calorie_intake
        return max_fat and max_protein and max_carbo and max_sodium and max_cholesterol and max_saturated_fat and makanan_bervariasi

    elif disease == "hypertension":
        max_fat =  breakfast["FatContent"] + lunch["FatContent"] + dinner["FatContent"] < 0.25 * calorie_intake
        max_protein = breakfast["ProteinContent"] + lunch["ProteinContent"] + dinner["ProteinContent"] < 0.15 * calorie_intake
        max_carbo = breakfast["CarbohydrateContent"] + lunch["CarbohydrateContent"] + dinner["CarbohydrateContent"]  < 0.60 * calorie_intake
        max_sodium =  breakfast["SodiumContent"] + lunch["SodiumContent"] + dinner["SodiumContent"]< 1000
        max_saturated_fat = saturated_fat < 0.07 * calorie_intake
        return max_fat and max_protein and max_carbo and max_sodium and max_saturated_fat and makanan_bervariasi
 
    elif disease == "normal":
        max_fat = breakfast["FatContent"] + lunch["FatContent"] + dinner["FatContent"] < 0.25 * calorie_intake
        max_protein =  breakfast["ProteinContent"] + lunch["ProteinContent"] + dinner["ProteinContent"] < 0.15 * calorie_intake
        max_carbo = breakfast["CarbohydrateContent"] + lunch["CarbohydrateContent"] + dinner["CarbohydrateContent"]  < 0.60 * calorie_intake
        max_sodium = breakfast["SodiumContent"] + lunch["SodiumContent"] + dinner["SodiumContent"] < 2000
        return max_fat and max_protein and max_carbo and max_sodium and makanan_bervariasi
        
        

problem.addConstraint(lambda breakfast, lunch, dinner: filter_based_on_disease(breakfast, lunch, dinner, my_calorie_daily_intake, "diabetes", "Vegetable"),
                       ("breakfast", "lunch", "dinner"))

# problem.addConstraint(lambda breakfast: breakfast["kalori"] < 600 and breakfast["sodium"] < 100
#                       , ("breakfast"))

# problem.addConstraint(lambda lunch: lunch["kalori"] < 1200 and lunch["kolesterol"] < 600
#                       , ("lunch"))
meal_plan_recommendation = problem.getSolution()
print("breakfast: ", meal_plan_recommendation["breakfast"], " \n")
print("lunch: ", meal_plan_recommendation["lunch"], " \n")
print("dinner: ", meal_plan_recommendation["dinner"], " \n")


breakfast:  {'RecipeId': 14905, 'Name': 'Chicken and Vegetables with Balsamic Vinegar', 'AuthorId': 21752, 'AuthorName': 'Mirj2338', 'CookTime': 'PT30M', 'PrepTime': 'PT10M', 'TotalTime': 'PT40M', 'DatePublished': '2001-11-27T09:20:00Z', 'Description': 'Chicken with balsamic vinegar is heaven on a plate. This is a very easy dish that easily impresses. Serve with plain white rice.', 'Images': 'character(0)', 'RecipeCategory': 'Lunch/Snacks', 'Keywords': 'c("Chicken", "Poultry", "Vegetable", "Meat", "Healthy", "Kosher", "Free Of...", "Potluck", "< 60 Mins", "Stove Top")', 'RecipeIngredientQuantities': 'c("1", "6", "1", "3", "1/2", "2", "5", "1", "1/4", "1/2", "1/4", "1/4")', 'RecipeIngredientParts': 'olive oil boneless skinless chicken breast halves baby red potato shallots onions broth leeks carrots red bell pepper garlic powder salt pepper', 'AggregatedRating': 4.0, 'ReviewCount': 3.0, 'Calories': 408.1, 'FatContent': 8.5, 'SaturatedFatContent': 1.6, 'CholesterolContent': 113.3, 'Sodiu